In [158]:
# системные модули
import os
import warnings
import sys
from typing import Union
# работа с данными
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from datetime import datetime

warnings.filterwarnings("ignore")

In [2]:
dates = ['FLTDAT', 'DAIS']
dtypes = {'FLTNUM': 'int16', 'TARIF_RUB': 'float16'}
bb = pd.read_csv('F:\\SVO-AER_2018-01-01--2019-12-31BB.csv',
                 delimiter=';', dtype=dtypes, parse_dates=dates)
bb = bb.dropna(how='any', axis=0)
bb.drop_duplicates(inplace=True)
bb['DEPARTURE'] = bb['FLTDAT'] + pd.to_timedelta(bb['FLTTIME'])
bb = bb[['FLTDAT', 'DEPARTURE', 'FLTNUM']].drop_duplicates()
bb.head()

,FLTDAT,DEPARTURE,FLTNUM
0,2018-01-21,2018-01-21 13:35:00,1124
1,2018-01-10,2018-01-10 09:10:00,1122
2,2018-01-11,2018-01-11 09:10:00,1122
3,2018-01-26,2018-01-26 17:35:00,1140
4,2018-01-06,2018-01-06 09:10:00,1122


In [3]:
%%time
dates = ['SDAT_S', 'DD']
dtypes = {'FLT_NUM': 'int16', 'FCLCLD': 'int16', 'PASS_BK': 'int16','SA': 'int16',
          'AU': 'int16', 'PASS_DEP': 'int16', 'NS': 'int16', 'DTD': 'int16'}
hh = pd.read_csv('F:\\hh_for_profiles.csv', dtype=dtypes, parse_dates=dates)

Wall time: 1min 2s


In [156]:
def get_flight(fltnum: int, date: datetime, flttime: pd.Timedelta) -> (str, pd.DataFrame):
    """Получение данных рейса для восстановления спроса"""
    departure = date + flttime
    flight = bb[(bb['FLTNUM'] == fltnum) & (bb['DEPARTURE'] == departure)]
    print(f"Рейс:", fltnum, departure)
    flight = pd.merge(flight, hh, left_on=['FLTNUM', 'FLTDAT'], right_on=['FLT_NUM', 'DD'])
    flight = flight.dropna(how='any', axis=0)
    flight.drop(columns=['FLTDAT', 'FLT_NUM', 'DD'], inplace=True)
    flight.drop_duplicates(inplace=True)
    print(f"Данных: {flight.shape[0]:_}")
    return f"{fltnum}, {departure}", flight

def get_regression(data: pd.DataFrame, code: str) -> pd.DataFrame:
    """Получение коэффициентов линейной регрессии"""
    # фильрация для одного из классов продаж:
    data = data[data['SEG_CLASS_CODE'] == code]
    # получение певрого дня последнего цензурирования
    first_last_one = np.where(data['FCLCLD'] == 0)[0]
    if len(first_last_one) != 0:
        first_last_one = first_last_one[-1] + 1
        # первый день последнего цензурирования влияет на наклон регрессии
        data.loc[data['DTD'] == first_last_one, 'FCLCLD'] = -1
    # все дней продаж билетов:
    Ts = max(data['DTD'].unique())
    # если время продаж билетов < 20%:
    if len(data[data['FCLCLD'] == 0]) / len(data) < .2:
        return 0, 0, Ts
    # фильрация по времени нецензурирования:
    data = data[data['FCLCLD'] != 1]
    # числа для МНК:
    z = data['PASS_BK'].values
    t = data['DTD'].values.reshape(-1, 1)
    # если после фильра осталась 1 или 0 точек (линейная регрессия не существует):
    if z.size <= 1:
        return 0, 0, Ts
    # коэффициенты линейной ресгрессии:
    reg = LinearRegression().fit(t, z)
    B1, B0 = reg.coef_[0], reg.intercept_
    return B1, B0, Ts

def get_Ts_streak(B1: np.float64, B0: np.float64, Ts: np.int16) -> Union[np.float64, np.int16]:
    """Получение значения Т штрих"""
    if B1 <= 0:
        return Ts  # max(DTD)
    Ts_streak = -B0 / B1
    if Ts_streak < 0 or Ts_streak > Ts:
        return Ts  # max(DTD)
    return Ts_streak

def get_restored_demand(Ts_streak: Union[np.float64, np.int16], class_code: str):
    """Получение восстановленного спроса"""
    Tmax = 10
    flight_class = flight[flight['SEG_CLASS_CODE'] == class_code]
    flight_class = flight_class[['DTD', 'PASS_BK', 'PASS_DEP', 'FCLCLD']]
    flight_class = flight_class.sort_values(by=['DTD'], ascending=False).reset_index(drop=True)
    # первый день последнего раза закрытия продаж
    first_last_one = np.where(flight_class['FCLCLD'] == 0)[0]
    # случай, если продажи не были закрыты в 0 или -1 день (спрос уже восстановленный):
    restored_demand_fail = flight_class[flight_class['DTD'].isin([-1, 0])]
    if len(first_last_one) == 0:
        restored_demand_fail['restored'] = restored_demand_fail['PASS_BK']
        return restored_demand_fail
    # Если больше трети было последнее цензурирование
    if len(first_last_one) / len(flight_class) > .33:
        Ts_streak = flight_class['DTD'].max()
    first_last_one = first_last_one[-1] + 1
    # получения последних дней цензурирования
    restored_demand = flight_class[first_last_one:].reset_index(drop=True)
    # восстановление проходит только на последние Tmax дней от 0
    restored_demand = restored_demand[restored_demand['DTD'] <= Tmax]
    # случай, если продажи не были закрыты даже в -1 день (спрос уже восстановленный):
    if len(restored_demand) == 0:
        restored_demand_fail['restored'] = restored_demand_fail['PASS_BK']
        return restored_demand_fail
    # количество дней последнего цензурирования до вылета:
    Tc = max(restored_demand['DTD'].unique())
    # бронирования класса на момент последнего закрытия продаж:
    Zc = restored_demand[restored_demand['DTD'] == Tc]['PASS_BK']
    restored_demand['restored'] = restored_demand.apply(
        lambda x: (Zc * (Ts_streak - x['DTD']) ** 2) /
                  (     (Ts_streak - Tc) ** 2)
        , axis=1
    )
    restored_demand.rename_axis(class_code, axis="columns", inplace=True)
    return restored_demand

def get_ful_restoration(flight: pd.DataFrame) -> pd.DataFrame:
    """
    Получение восстановленного спроса по всем классам вылета в табличном виде на каждый день DTD,
    где в -1 день отображается фактическое кол-во пассажиров, с 0 до TMAX - восстановленный спрос,
    с TMAX + 1 - кол-во фактических бронирований.
    """
    restored = dict()
    for class_code in flight['SEG_CLASS_CODE'].unique():
        B1, B0, Ts = get_regression(flight, class_code)
        Ts_streak = get_Ts_streak(B1, B0, Ts)
        flight_class = get_restored_demand(Ts_streak, class_code)
        
        flight_cp = flight[flight['SEG_CLASS_CODE'] == class_code][['PASS_BK', 'DTD']]
        # фактическое кол-во вылетевших людей
        pass_dp = flight_class[flight_class['DTD'] == -1]['PASS_DEP'].values[0]
        # восстановленный спрос
        pass_bk_restored = flight_class[flight_class['DTD'] != -1]
        max_dtd = pass_bk_restored['DTD'].max()
        # получение восстановленного спроса с 0 до Tmax
        pass_bk_restored = pass_bk_restored['restored'].values.tolist()[::-1]
        restored.setdefault(class_code, [pass_dp]).extend(pass_bk_restored)
        # фактическое количество бронирования с Tmax + 1 дня до Ts
        pass_bk_unrestored = flight_cp.loc[flight_cp['DTD'] > max_dtd]['PASS_BK'].values.tolist()[::-1]
        restored[class_code].extend(pass_bk_unrestored)
    restored = pd.DataFrame(restored)
    return restored

In [6]:
%%time
flight_name, flight = get_flight(1122,
                                 datetime.strptime('2018-01-06', '%Y-%m-%d'),
                                 pd.Timedelta('09:10:00'))

Рейс: 1122 2018-01-06 09:10:00
Данных: 4_158
Wall time: 2.1 s


In [157]:
get_ful_restoration(flight)

,J,C,D,I,Z,O,Y,B,M,U,...,H,L,X,Q,T,E,N,R,G,V
0,0,0,3,0,0.0,6.000000,0,0,0.0,0.0,...,5.00000,5.00000,13.000000,4.000000,22.000000,22.000000,83.000000,3.000000,0,0
1,0,0,3,0,0.0,6.055944,0,0,0.0,0.0,...,5.04662,5.04662,14.262905,4.075116,26.737552,23.055008,93.640832,3.298332,0,0
2,0,0,3,0,0.0,6.000000,0,1,0.0,0.0,...,5.00000,5.00000,14.131147,4.037471,26.490555,22.842030,92.775794,3.267862,0,0
3,0,0,3,0,0.0,6.000000,0,0,0.0,0.0,...,4.00000,2.00000,14.000000,4.000000,26.244705,22.630040,91.914770,3.237534,0,0
4,2,0,3,0,0.0,4.000000,0,0,0.0,0.0,...,4.00000,2.00000,8.000000,2.000000,26.000000,22.419038,91.057761,3.207348,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,2,0,0,0,0.0,0.000000,0,0,0.0,0.0,...,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0
185,2,0,0,0,0.0,0.000000,0,0,0.0,0.0,...,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0
186,2,0,0,0,0.0,0.000000,0,0,0.0,0.0,...,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0
187,2,0,0,0,0.0,0.000000,0,0,0.0,0.0,...,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0
